# IFKS 2025 Analyse - Drie Gebroeders

Dit notebook analyseert de zeilprestaties van **Drie Gebroeders** tijdens de IFKS 2025.

## Analyses
1. Data laden en opschonen (alleen race-tijd, geen pre-start/haven)
2. Gemiddelde snelheid per wedstrijd
3. Hoek met de wind (True Wind Angle)
4. Snelheid vs windhoek (polar-achtige analyse)
5. Vergelijking met concurrenten
6. VMG analyse (Velocity Made Good)

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuratie
TEAM_NAAM = "Drie Gebroeders"
DATA_DIR = Path("Data")

# Kleuren voor visualisatie
TEAM_COLOR = '#1f77b4'
OTHER_COLOR = '#cccccc'

## 1. Data Laden en Opschonen

We laden alle wedstrijddata en filteren op:
- Alleen data tussen `starttime` en `endtime` van de wedstrijd
- Verwijderen van stilliggende periodes (snelheid < drempelwaarde)
- Verwijderen van uitschieters (onrealistische snelheden)

In [ ]:
def load_race_data(filepath):
    """Laad een wedstrijdbestand en extraheer metadata."""
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data

def ship_to_dataframe(ship_data, race_name):
    """Converteer ship data naar een DataFrame."""
    df = pd.DataFrame({
        'timestamp': ship_data['stamp'],
        'lat': ship_data['lat'],
        'lon': ship_data['lon'],
        'speed': ship_data['speed'],
        'course': ship_data['course']
    })
    df['ship_name'] = ship_data['name']
    df['race'] = race_name
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
    return df

def wind_to_dataframe(wind_data):
    """Converteer wind data naar een DataFrame."""
    df = pd.DataFrame({
        'timestamp': wind_data['stamp'],
        'lat': wind_data['lat'],
        'lon': wind_data['lon'],
        'wind_speed': wind_data['speed'],
        'wind_direction': wind_data['course']
    })
    df['station'] = wind_data['name']
    return df

def clean_race_data(df, starttime, endtime, speed_threshold=5, max_speed=200):
    """Filter data op race-tijd en verwijder onrealistische waarden."""
    # Filter op race-tijd (start tot finish)
    df_clean = df[(df['timestamp'] >= starttime) & (df['timestamp'] <= endtime)].copy()
    
    # Verwijder stilliggende periodes (haven/pre-start)
    df_clean = df_clean[df_clean['speed'] >= speed_threshold]
    
    # Verwijder uitschieters (onrealistische snelheden)
    df_clean = df_clean[df_clean['speed'] <= max_speed]
    
    return df_clean

# Laad alle wedstrijden
race_files = sorted(DATA_DIR.glob("B-Match*.json"))
print(f"Gevonden wedstrijden: {len(race_files)}")

all_races = {}
all_wind = {}

for filepath in race_files:
    race_name = filepath.stem.replace('B-', '')
    data = load_race_data(filepath)
    
    starttime = data.get('starttime', 0)
    endtime = data.get('endtime', 0)
    
    print(f"\n{race_name}:")
    print(f"  Start: {datetime.fromtimestamp(starttime)}")
    print(f"  Einde: {datetime.fromtimestamp(endtime)}")
    print(f"  Duur: {(endtime - starttime) / 60:.1f} minuten")
    
    # Verwerk ships
    ships_df = []
    for ship in data.get('shiptracks', []):
        df = ship_to_dataframe(ship, race_name)
        df_clean = clean_race_data(df, starttime, endtime)
        if len(df_clean) > 0:
            ships_df.append(df_clean)
    
    all_races[race_name] = pd.concat(ships_df, ignore_index=True) if ships_df else pd.DataFrame()
    
    # Verwerk wind
    wind_df = []
    for wind in data.get('windtracks', []):
        df = wind_to_dataframe(wind)
        # Filter wind ook op race-tijd
        df = df[(df['timestamp'] >= starttime) & (df['timestamp'] <= endtime)]
        if len(df) > 0:
            wind_df.append(df)
    
    all_wind[race_name] = pd.concat(wind_df, ignore_index=True) if wind_df else pd.DataFrame()
    
    # Stats voor Drie Gebroeders
    team_data = all_races[race_name][all_races[race_name]['ship_name'] == TEAM_NAAM]
    print(f"  {TEAM_NAAM}: {len(team_data)} datapunten na cleaning")

In [ ]:
# Combineer alle data
df_all = pd.concat(all_races.values(), ignore_index=True)
df_team = df_all[df_all['ship_name'] == TEAM_NAAM].copy()

print(f"Totaal datapunten alle schepen: {len(df_all):,}")
print(f"Totaal datapunten {TEAM_NAAM}: {len(df_team):,}")
print(f"\nSchepen in dataset: {df_all['ship_name'].nunique()}")
print(f"Wedstrijden: {df_all['race'].nunique()}")

## 2. Gemiddelde Snelheid per Wedstrijd

Vergelijk de gemiddelde snelheid van Drie Gebroeders met het veld.

In [ ]:
# Bereken gemiddelde snelheid per schip per wedstrijd
speed_stats = df_all.groupby(['race', 'ship_name'])['speed'].agg(['mean', 'max', 'std']).reset_index()
speed_stats.columns = ['race', 'ship_name', 'avg_speed', 'max_speed', 'std_speed']

# Scheid team en anderen
team_speeds = speed_stats[speed_stats['ship_name'] == TEAM_NAAM].set_index('race')
fleet_avg = speed_stats.groupby('race')['avg_speed'].mean()

# Plot
fig, ax = plt.subplots(figsize=(12, 6))

races = sorted(df_all['race'].unique())
x = range(len(races))

# Fleet gemiddelde
fleet_values = [fleet_avg.get(r, 0) for r in races]
ax.bar([i - 0.2 for i in x], fleet_values, 0.4, label='Vloot gemiddelde', color=OTHER_COLOR)

# Drie Gebroeders
team_values = [team_speeds.loc[r, 'avg_speed'] if r in team_speeds.index else 0 for r in races]
ax.bar([i + 0.2 for i in x], team_values, 0.4, label=TEAM_NAAM, color=TEAM_COLOR)

ax.set_xlabel('Wedstrijd')
ax.set_ylabel('Gemiddelde snelheid')
ax.set_title(f'Gemiddelde Snelheid per Wedstrijd: {TEAM_NAAM} vs Vloot')
ax.set_xticks(x)
ax.set_xticklabels([r.replace('Match', 'M') for r in races], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Numerieke samenvatting
print("\nSnelheidsstatistieken per wedstrijd:")
print("=" * 60)
for race in races:
    team_avg = team_speeds.loc[race, 'avg_speed'] if race in team_speeds.index else 0
    team_max = team_speeds.loc[race, 'max_speed'] if race in team_speeds.index else 0
    fleet = fleet_avg.get(race, 0)
    diff = team_avg - fleet
    diff_pct = (diff / fleet * 100) if fleet > 0 else 0
    print(f"{race:20} | Team: {team_avg:5.1f} | Vloot: {fleet:5.1f} | Verschil: {diff:+5.1f} ({diff_pct:+.1f}%)")

## 3. Hoek met de Wind (True Wind Angle)

Bereken de hoek tussen de koers van het schip en de windrichting.

- **0° / 180°**: Recht tegen/met de wind in
- **45°**: Aan de wind (kruisen)
- **90°**: Halve wind
- **135°**: Ruime wind
- **180°**: Voor de wind

In [ ]:
def calculate_twa(course, wind_direction):
    """Bereken True Wind Angle (0-180 graden)."""
    # Verschil tussen koers en windrichting
    diff = abs(course - wind_direction)
    # Normaliseer naar 0-180
    if diff > 180:
        diff = 360 - diff
    return diff

def get_avg_wind_for_timestamp(wind_df, timestamp, window=60):
    """Haal gemiddelde wind op rond een timestamp."""
    mask = (wind_df['timestamp'] >= timestamp - window) & (wind_df['timestamp'] <= timestamp + window)
    subset = wind_df[mask]
    if len(subset) > 0:
        # Circulair gemiddelde voor windrichting
        directions = np.radians(subset['wind_direction'])
        avg_dir = np.degrees(np.arctan2(np.mean(np.sin(directions)), np.mean(np.cos(directions)))) % 360
        return subset['wind_speed'].mean(), avg_dir
    return np.nan, np.nan

# Bereken TWA voor Drie Gebroeders in alle wedstrijden
twa_data = []

for race_name in all_races.keys():
    race_df = all_races[race_name]
    wind_df = all_wind[race_name]
    
    if len(wind_df) == 0:
        print(f"Geen winddata voor {race_name}")
        continue
    
    team_race = race_df[race_df['ship_name'] == TEAM_NAAM].copy()
    
    # Bereken gemiddelde windrichting per timestamp (simplified)
    avg_wind_dir = wind_df.groupby('timestamp')['wind_direction'].apply(
        lambda x: np.degrees(np.arctan2(np.mean(np.sin(np.radians(x))), np.mean(np.cos(np.radians(x))))) % 360
    )
    avg_wind_speed = wind_df.groupby('timestamp')['wind_speed'].mean()
    
    for _, row in team_race.iterrows():
        # Vind dichtstbijzijnde windmeting
        closest_ts = avg_wind_dir.index[np.abs(avg_wind_dir.index - row['timestamp']).argmin()]
        wind_dir = avg_wind_dir[closest_ts]
        wind_spd = avg_wind_speed.get(closest_ts, np.nan)
        
        twa = calculate_twa(row['course'], wind_dir)
        
        twa_data.append({
            'race': race_name,
            'timestamp': row['timestamp'],
            'speed': row['speed'],
            'course': row['course'],
            'wind_direction': wind_dir,
            'wind_speed': wind_spd,
            'twa': twa
        })

df_twa = pd.DataFrame(twa_data)
print(f"TWA berekend voor {len(df_twa):,} datapunten")

In [ ]:
# Visualiseer TWA distributie
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram van TWA
ax1 = axes[0]
ax1.hist(df_twa['twa'], bins=36, range=(0, 180), color=TEAM_COLOR, edgecolor='white', alpha=0.7)
ax1.axvline(45, color='red', linestyle='--', label='Aan de wind (45°)')
ax1.axvline(90, color='orange', linestyle='--', label='Halve wind (90°)')
ax1.axvline(135, color='green', linestyle='--', label='Ruime wind (135°)')
ax1.set_xlabel('True Wind Angle (graden)')
ax1.set_ylabel('Frequentie')
ax1.set_title(f'Verdeling Windhoek - {TEAM_NAAM}')
ax1.legend()

# TWA per wedstrijd
ax2 = axes[1]
df_twa.boxplot(column='twa', by='race', ax=ax2)
ax2.set_xlabel('Wedstrijd')
ax2.set_ylabel('True Wind Angle (graden)')
ax2.set_title(f'Windhoek per Wedstrijd - {TEAM_NAAM}')
plt.suptitle('')

plt.tight_layout()
plt.show()

# Statistieken
print("\nTWA Statistieken:")
print(f"Gemiddelde TWA: {df_twa['twa'].mean():.1f}°")
print(f"Mediaan TWA: {df_twa['twa'].median():.1f}°")
print(f"\nTijd per windhoek categorie:")
categories = [
    ('Aan de wind (0-60°)', (0, 60)),
    ('Halve wind (60-120°)', (60, 120)),
    ('Ruime wind (120-180°)', (120, 180))
]
for name, (low, high) in categories:
    pct = ((df_twa['twa'] >= low) & (df_twa['twa'] < high)).mean() * 100
    print(f"  {name}: {pct:.1f}%")

## 4. Snelheid vs Windhoek (Polar Analyse)

Hoe snel vaart Drie Gebroeders bij verschillende windhoeken?

In [ ]:
# Bin TWA en bereken gemiddelde snelheid per bin
df_twa['twa_bin'] = pd.cut(df_twa['twa'], bins=range(0, 190, 10), labels=range(5, 185, 10))
polar_data = df_twa.groupby('twa_bin', observed=True)['speed'].agg(['mean', 'std', 'count']).reset_index()
polar_data.columns = ['twa', 'avg_speed', 'std_speed', 'count']
polar_data['twa'] = polar_data['twa'].astype(float)

# Filter bins met weinig data
polar_data = polar_data[polar_data['count'] >= 10]

# Plot polar diagram
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Lineaire plot
ax1 = axes[0]
ax1.fill_between(polar_data['twa'], 
                  polar_data['avg_speed'] - polar_data['std_speed'],
                  polar_data['avg_speed'] + polar_data['std_speed'],
                  alpha=0.3, color=TEAM_COLOR)
ax1.plot(polar_data['twa'], polar_data['avg_speed'], 'o-', color=TEAM_COLOR, linewidth=2)
ax1.set_xlabel('True Wind Angle (graden)')
ax1.set_ylabel('Gemiddelde Snelheid')
ax1.set_title(f'Snelheid vs Windhoek - {TEAM_NAAM}')
ax1.grid(alpha=0.3)
ax1.set_xlim(0, 180)

# Polar plot
ax2 = axes[1]
ax2 = plt.subplot(122, projection='polar')
# Converteer naar radialen en spiegel voor beide kanten
theta = np.radians(polar_data['twa'])
r = polar_data['avg_speed']

# Plot beide kanten (symmetrisch)
ax2.plot(theta, r, 'o-', color=TEAM_COLOR, linewidth=2, label='Stuurboord')
ax2.plot(-theta, r, 'o-', color=TEAM_COLOR, linewidth=2, alpha=0.5, label='Bakboord')

ax2.set_theta_zero_location('N')  # Wind komt van boven
ax2.set_theta_direction(-1)  # Klokwijze
ax2.set_thetamin(-180)
ax2.set_thetamax(180)
ax2.set_title(f'Polar Diagram - {TEAM_NAAM}', pad=20)

plt.tight_layout()
plt.show()

# Beste hoeken
print("\nOptimale windhoeken:")
print(f"Hoogste gemiddelde snelheid: {polar_data['avg_speed'].max():.1f} bij {polar_data.loc[polar_data['avg_speed'].idxmax(), 'twa']:.0f}°")
print(f"\nSnelheid per windhoek:")
for _, row in polar_data.iterrows():
    print(f"  {row['twa']:3.0f}°: {row['avg_speed']:5.1f} (n={row['count']:.0f})")

## 5. Vergelijking met Concurrenten

Hoe presteert Drie Gebroeders ten opzichte van de top 5?

In [ ]:
# Bereken overall gemiddelde snelheid per schip
overall_speed = df_all.groupby('ship_name')['speed'].agg(['mean', 'max', 'std', 'count']).reset_index()
overall_speed.columns = ['ship_name', 'avg_speed', 'max_speed', 'std_speed', 'data_points']
overall_speed = overall_speed.sort_values('avg_speed', ascending=False)

# Ranking
overall_speed['rank'] = range(1, len(overall_speed) + 1)

# Highlight Drie Gebroeders
team_rank = overall_speed[overall_speed['ship_name'] == TEAM_NAAM]['rank'].values[0]

# Plot
fig, ax = plt.subplots(figsize=(12, 8))

colors = [TEAM_COLOR if name == TEAM_NAAM else OTHER_COLOR for name in overall_speed['ship_name']]
bars = ax.barh(overall_speed['ship_name'], overall_speed['avg_speed'], color=colors)

# Annotatie voor Drie Gebroeders
team_speed = overall_speed[overall_speed['ship_name'] == TEAM_NAAM]['avg_speed'].values[0]
ax.axvline(team_speed, color='red', linestyle='--', alpha=0.5)

ax.set_xlabel('Gemiddelde Snelheid')
ax.set_ylabel('Schip')
ax.set_title(f'Snelheidsranking - {TEAM_NAAM} staat #{team_rank}')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Tabel
print("\nTop 10 + Drie Gebroeders:")
print("=" * 70)
top_10 = overall_speed.head(10)
team_row = overall_speed[overall_speed['ship_name'] == TEAM_NAAM]
if team_rank > 10:
    display_df = pd.concat([top_10, team_row])
else:
    display_df = top_10

for _, row in display_df.iterrows():
    marker = ">>>" if row['ship_name'] == TEAM_NAAM else "   "
    print(f"{marker} #{row['rank']:2.0f} {row['ship_name']:20} | Gem: {row['avg_speed']:5.1f} | Max: {row['max_speed']:5.0f}")

In [ ]:
# Ranking per wedstrijd
race_rankings = []

for race in df_all['race'].unique():
    race_speeds = df_all[df_all['race'] == race].groupby('ship_name')['speed'].mean().sort_values(ascending=False)
    for rank, (ship, speed) in enumerate(race_speeds.items(), 1):
        race_rankings.append({
            'race': race,
            'ship_name': ship,
            'avg_speed': speed,
            'rank': rank
        })

df_rankings = pd.DataFrame(race_rankings)

# Drie Gebroeders ranking per wedstrijd
team_rankings = df_rankings[df_rankings['ship_name'] == TEAM_NAAM].sort_values('race')

# Plot
fig, ax = plt.subplots(figsize=(12, 5))

races = sorted(team_rankings['race'].unique())
ranks = [team_rankings[team_rankings['race'] == r]['rank'].values[0] for r in races]

ax.plot(races, ranks, 'o-', color=TEAM_COLOR, linewidth=2, markersize=10)
ax.axhline(y=np.mean(ranks), color='red', linestyle='--', alpha=0.5, label=f'Gemiddelde: {np.mean(ranks):.1f}')

ax.set_xlabel('Wedstrijd')
ax.set_ylabel('Ranking (lager is beter)')
ax.set_title(f'Ranking per Wedstrijd - {TEAM_NAAM}')
ax.invert_yaxis()  # Lager is beter
ax.set_ylim(16.5, 0.5)
ax.set_yticks(range(1, 17))
ax.legend()
ax.grid(alpha=0.3)

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print(f"\nRanking {TEAM_NAAM} per wedstrijd:")
for race, rank in zip(races, ranks):
    total_ships = len(df_rankings[df_rankings['race'] == race])
    print(f"  {race}: #{rank} van {total_ships}")
print(f"\nGemiddelde ranking: {np.mean(ranks):.1f}")
print(f"Beste prestatie: #{min(ranks)} ({races[ranks.index(min(ranks))]})")
print(f"Slechtste prestatie: #{max(ranks)} ({races[ranks.index(max(ranks))]})")

## 6. VMG Analyse (Velocity Made Good)

VMG meet hoe snel je daadwerkelijk richting je doel vaart (rekening houdend met de windhoek).

- **Upwind VMG**: Snelheid * cos(TWA) - hoe goed je tegen de wind in komt
- **Downwind VMG**: Snelheid * cos(180 - TWA) - hoe goed je met de wind mee komt

In [ ]:
# Bereken VMG
df_twa['vmg_upwind'] = df_twa['speed'] * np.cos(np.radians(df_twa['twa']))
df_twa['vmg_downwind'] = df_twa['speed'] * np.cos(np.radians(180 - df_twa['twa']))

# Categoriseer in upwind/downwind
df_twa['sailing_mode'] = np.where(df_twa['twa'] < 90, 'Upwind', 'Downwind')

# VMG per windhoek bin
vmg_by_twa = df_twa.groupby('twa_bin', observed=True).agg({
    'vmg_upwind': 'mean',
    'vmg_downwind': 'mean',
    'speed': 'mean',
    'twa': 'count'
}).reset_index()
vmg_by_twa.columns = ['twa_bin', 'vmg_upwind', 'vmg_downwind', 'boat_speed', 'count']
vmg_by_twa['twa'] = vmg_by_twa['twa_bin'].astype(float)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# VMG vs TWA
ax1 = axes[0]
upwind_data = vmg_by_twa[vmg_by_twa['twa'] < 90]
downwind_data = vmg_by_twa[vmg_by_twa['twa'] >= 90]

ax1.plot(upwind_data['twa'], upwind_data['vmg_upwind'], 'o-', color='blue', label='Upwind VMG', linewidth=2)
ax1.plot(downwind_data['twa'], downwind_data['vmg_downwind'], 'o-', color='red', label='Downwind VMG', linewidth=2)
ax1.axhline(0, color='gray', linestyle='-', alpha=0.3)
ax1.set_xlabel('True Wind Angle (graden)')
ax1.set_ylabel('VMG')
ax1.set_title(f'VMG vs Windhoek - {TEAM_NAAM}')
ax1.legend()
ax1.grid(alpha=0.3)

# VMG distributie
ax2 = axes[1]
upwind = df_twa[df_twa['sailing_mode'] == 'Upwind']['vmg_upwind']
downwind = df_twa[df_twa['sailing_mode'] == 'Downwind']['vmg_downwind']

ax2.hist(upwind, bins=30, alpha=0.7, label=f'Upwind VMG (gem: {upwind.mean():.1f})', color='blue')
ax2.hist(downwind, bins=30, alpha=0.7, label=f'Downwind VMG (gem: {downwind.mean():.1f})', color='red')
ax2.set_xlabel('VMG')
ax2.set_ylabel('Frequentie')
ax2.set_title(f'VMG Distributie - {TEAM_NAAM}')
ax2.legend()

plt.tight_layout()
plt.show()

# Optimale hoeken
best_upwind_idx = upwind_data['vmg_upwind'].idxmax()
best_downwind_idx = downwind_data['vmg_downwind'].idxmax()

print("\nOptimale VMG hoeken:")
print(f"Beste upwind hoek: {vmg_by_twa.loc[best_upwind_idx, 'twa']:.0f}° (VMG: {vmg_by_twa.loc[best_upwind_idx, 'vmg_upwind']:.1f})")
print(f"Beste downwind hoek: {vmg_by_twa.loc[best_downwind_idx, 'twa']:.0f}° (VMG: {vmg_by_twa.loc[best_downwind_idx, 'vmg_downwind']:.1f})")

## 7. Samenvatting

In [ ]:
print("=" * 70)
print(f"SAMENVATTING IFKS 2025 - {TEAM_NAAM}")
print("=" * 70)

print(f"\n📊 ALGEMEEN")
print(f"  Wedstrijden gevaren: {len(all_races)}")
print(f"  Totaal datapunten: {len(df_team):,}")

print(f"\n⚡ SNELHEID")
print(f"  Gemiddelde snelheid: {df_team['speed'].mean():.1f}")
print(f"  Maximum snelheid: {df_team['speed'].max():.0f}")
print(f"  Overall ranking: #{team_rank} van {len(overall_speed)}")

print(f"\n🌬️ WINDHOEK")
print(f"  Gemiddelde TWA: {df_twa['twa'].mean():.1f}°")
print(f"  Meest voorkomende hoek: {df_twa['twa_bin'].mode().values[0]}°")
print(f"  Upwind tijd: {(df_twa['twa'] < 90).mean() * 100:.1f}%")
print(f"  Downwind tijd: {(df_twa['twa'] >= 90).mean() * 100:.1f}%")

print(f"\n🎯 VMG")
print(f"  Beste upwind VMG hoek: {vmg_by_twa.loc[best_upwind_idx, 'twa']:.0f}°")
print(f"  Beste downwind VMG hoek: {vmg_by_twa.loc[best_downwind_idx, 'twa']:.0f}°")

print(f"\n📈 PRESTATIE PER WEDSTRIJD")
for race, rank in zip(races, ranks):
    emoji = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉" if rank == 3 else "  "
    print(f"  {emoji} {race}: #{rank}")

print("\n" + "=" * 70)

---

## Mogelijke Vervolganalyses

1. **Manoeuvre-analyse**: Identificeer overstag/gijpen momenten en analyseer snelheidsverlies
2. **Start-analyse**: Hoe goed is de start? Positie en snelheid bij de startlijn
3. **Baan-analyse**: Welke routes werden gekozen? Vergelijk met winnaars
4. **Windshift analyse**: Hoe goed wordt gereageerd op windveranderingen?
5. **Performance per windsterkte**: Beter in lichte of zware wind?
6. **Head-to-head vergelijking**: Directe vergelijking met specifieke concurrenten